运行以下程序需先安装图像处理第三方库pillow,pip install PIL

In [ ]:
import os
import struct
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

#本题以MNIST为数据集，MNIST是一个手写数字0-9的数据集，有60000个训练样本和10000个测试样本
#装入训练图像及标签到变量x_train, y_train
kind='train'
path=r'.\data\mnist'
labels_path = os.path.join(path, '%s-labels-idx1-ubyte' % kind)#训练标签路径
images_path = os.path.join(path, '%s-images-idx3-ubyte' % kind)#训练图像路径

with open(labels_path, 'rb') as lbpath:#打开训练标签文件并读取数据
    magic,n  = struct.unpack('>2I',lbpath.read(8))
    labels = np.fromfile(lbpath, dtype=np.uint8)

with open(images_path, 'rb') as imgpath:#打开训练图像文件并读取数据
    magic, num, rows, cols = struct.unpack(">4I",imgpath.read(16))
    images = np.fromfile(imgpath, dtype=np.uint8).reshape(len(labels), 784)/255.0

x_train, y_train =images,labels

#装入测试图像及标签到变量x_test, y_test
kind='t10k'
path=r'.\data\mnist'
labels_path = os.path.join(path, '%s-labels-idx1-ubyte' % kind)#测试标签路径
images_path = os.path.join(path, '%s-images-idx3-ubyte' % kind)#训练图像路径

with open(labels_path, 'rb') as lbpath:#打开测试标签文件并读取数据
    magic,n  = struct.unpack('>2I',lbpath.read(8))
    labels = np.fromfile(lbpath, dtype=np.uint8)

with open(images_path, 'rb') as imgpath:#打开测试图像文件并读取数据
    magic, num, rows, cols = struct.unpack(">4I",imgpath.read(16))
    images = np.fromfile(imgpath, dtype=np.uint8).reshape(len(labels), 784)/255.0

x_test, y_test =images,labels

#print(x_train.shape)

'''
#查看样本数据 抽取10张手写的数字图片
fig, ax = plt.subplots(nrows=2, ncols=5, sharex=True, sharey=True,)
ax = ax.flatten()
#0到9的数字中每个取一个
for i in range(10):
    img = x_train[y_train==i][0].reshape(28, 28)
    ax[i].imshow(img, cmap='Greys', interpolation='nearest')
   
#不显示坐标轴
plt.xticks([])
plt.yticks([])
#plt.show()
'''
import math
import random
 
def make_matrix(m,n,fill=0.0):#创建一个指定大小的矩阵
    mat = []
    for i in range(m):
        mat.append([fill]*n)
    return mat
 
def sigmoid(x):#定义sigmoid函数
    return 1.0/(1.0+math.exp(-x))

def sigmoid_derivate(x):#sigmoid函数的导数
    return sigmoid(x)*(1-sigmoid(x)) 
 
class BPNN:
    def __init__(self):#初始化变量
        self.input_n = 0
        self.hidden_n = 0
        self.output_n = 0
        self.input_cells = []#输入层加权和--I/O
        self.hidden_cells = []#隐含层加权和--I
        self.hidden_cells_outs=[]#sigmoid(隐含层加权和)--输出O
        self.output_cells = []#输出层加权和--I
        self.output_cells_outs=[]#sigmoid(输出层加权和)--输出O
        self.input_weights = []#输入层-隐含层权值矩阵
        self.output_weights = []#隐含层-输出层权值矩阵
    
    #三个列表维护：输入层，隐含层，输出层神经元
    def setup(self,ni,nh,no):
        self.input_n = ni+1   #输入层+偏置项
        self.hidden_n = nh+1  #隐含层+偏置项
        self.output_n = no    #输出层 
        #初始化神经元
        self.input_cells = [1.0]*self.input_n
        self.hidden_cells= [1.0]*self.hidden_n
        self.hidden_cells_outs= [1.0]*self.hidden_n
        self.output_cells= [1.0]*self.output_n
        self.output_cells_outs= [1.0]*self.output_n
 
        #初始化连接边的边权
        self.input_weights = make_matrix(self.input_n,self.hidden_n) #邻接矩阵存边权：输入层->隐藏层
        self.output_weights = make_matrix(self.hidden_n,self.output_n) #邻接矩阵存边权：隐藏层->输出层 
        #随机初始化边权：为了反向传导做准备--->随机初始化的目的是使对称失效
        #np.random.seed(2)
        self.input_weights=np.random.randn(self.input_n,self.hidden_n)
        self.output_weights=np.random.randn(self.hidden_n,self.output_n)
         
    #输出预测值
    def predict(self,inputs):
        #对输入层进行操作转化样本
        for i in range(self.input_n-1):
            self.input_cells[i] = inputs[i] #n个样本从0~n-1
        #计算隐藏层的输出，每个节点最终的输出值就是权值*节点值的加权和
        for h in range(self.hidden_n-1):
            total = 0.0
            for i in range(self.input_n):
                total+=self.input_cells[i]*self.input_weights[i][h]#隐层结点加权和
            # 此处为何是先i再j，以隐含层节点做大循环，输入样本为小循环，
            #是为了每一个隐藏节点计算一个输出值，传输到下一层
            self.hidden_cells[h] = total #存储隐层结点加权和
            self.hidden_cells_outs[h]= sigmoid(total)#存储隐层结点输出值
        #计算输出层的输出
        for o in range(self.output_n):
            total = 0.0
            for h in range(self.hidden_n):
                total+=self.hidden_cells_outs[h]*self.output_weights[h][o]#输出层结点加权和
            self.output_cells[o] = total #存储输出层结点加权和
            self.output_cells_outs[o] = sigmoid(total)#存储输出层结点输出值
        return #self.output_cells_outs[:]  #最后输出层的结果返回
 
    #反向传播算法：调用预测函数，根据反向传播获取权重后前向预测，将结果与实际结果返回比较误差
    def back_propagate(self,case,label,learn_rate): 
        #对输入样本做预测
        self.predict(case) #对实例进行预测
        output_deltas = [0.0]*self.output_n #初始化输出层delta值
        for o in range(self.output_n):
            error = label[o] - self.output_cells_outs[o] #期望结果（即真实值）和预测结果的误差
            output_deltas[o]= sigmoid_derivate(self.output_cells[o])*error#输出层delta=(期望值-预测值)*输出层激活函数的导数
        #隐含层误差
        hidden_deltas = [0.0]*self.hidden_n #初始化隐含层delta值
        for h in range(self.hidden_n):
            error = 0.0
            for o in range(self.output_n):
                error+=output_deltas[o]*self.output_weights[h][o]
            hidden_deltas[h] = sigmoid_derivate(self.hidden_cells[h])*error#隐含层delta
        #反向传播算法求W
        #更新隐藏层->输出权重
        for h in range(self.hidden_n):
            for o in range(self.output_n):
                change = output_deltas[o]*self.hidden_cells_outs[h]
                #调整权重：上一层每个节点的权重学习*变化
                self.output_weights[h][o] += -1.0*learn_rate*change  
        #更新输入->隐藏层的权重
        for i in range(self.input_n):
            for h in range(self.hidden_n):
                change = hidden_deltas[h]*self.input_cells[i]
                self.input_weights[i][h] += -1.0*learn_rate*change 
        return 
 
    def train(self,cases,labels,limit=10000,learn_rate=0.01):
        for i in range(limit): #设置迭代次数
            error = 0.0
            for j in range(len(cases)):#对输入层进行访问
                case = cases[j]
                label = labels[j]
                label_one_hot=np.zeros((10,1))
                #动态修改学习率
                #if j%2000==0:
                    #learn_rate=learn_rate*0.99
                #转换为one_hot编码
                label_one_hot[int(label)]=1
                self.back_propagate(case,label_one_hot,learn_rate)
        #print("学习总体误差：",error) #误差
 
def study(train_cases,train_labels,test_cases,test_labels): #学习       
        B=BPNN()
        B.setup(784,100,10) #初始化神经网络：输入层，隐藏层，输出层元素个数
        B.train(train_cases,train_labels,1,0.01)#样例，标签，迭代次数，学习率
        count=0#用于记录预测正确结果的个数
        for j in range(len(test_cases)):#对输入层进行访问
            case =test_cases[j]
            label=test_labels[j]
            test_value=B.predict(case) #对实例进行预测
            #print(test_value)
            test_result=np.argmax(test_value)#获得预测的one_hot编码值
            #print(test_result)
            if test_result==int(label):
                count+=1
        print("预测准确率：",count/len(test_cases))

if __name__ == '__main__':
    train_cases =x_train[:1000,:] #训练样例
    train_labels = y_train[:1000] #训练标签
    test_cases=x_test[:10,:]#测试样本
    test_labels=y_test[:10]#测试标签
    study(train_cases,train_labels,test_cases,test_labels)#学习+测试

预测准确率： 0.1
